In [1]:
import sys
sys.path.append('/home/ec2-user/SageMaker')
from toniutils import dirtoni as dirt
from toniutils import printoni

<h2>Load Data</h2>

In [2]:
from script import load

In [3]:
path = 'engineered_data/experiment-1/'
X_train, X_val, Y_train, Y_val = [load(path + i + '.pkl') for i in ('X_train', 'X_val', 'Y_train', 'Y_val')]

<h2>Train Model</h2>

In [4]:
import tensorflow as tf

In [15]:
from tensorflow import float32 as fl32
from tensorflow import random_normal as rnorm
from tensorflow import matmul, add
from tensorflow.nn import relu,dropout, tanh
from tensorflow.initializers import he_uniform, glorot_uniform
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 317502814057370109, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 9789628448623312100
 physical_device_desc: "device: XLA_CPU device"]

In [19]:
help(glorot_uniform)

Help on class GlorotUniform in module tensorflow.python.ops.init_ops:

class GlorotUniform(VarianceScaling)
 |  The Glorot uniform initializer, also called Xavier uniform initializer.
 |  
 |  It draws samples from a uniform distribution within [-limit, limit]
 |  where `limit` is `sqrt(6 / (fan_in + fan_out))`
 |  where `fan_in` is the number of input units in the weight tensor
 |  and `fan_out` is the number of output units in the weight tensor.
 |  
 |  Args:
 |    seed: A Python integer. Used to create random seeds. See
 |      `tf.set_random_seed`
 |      for behavior.
 |    dtype: Default data type, used if no `dtype` argument is provided when
 |      calling the initializer. Only floating point types are supported.
 |  
 |  References:
 |      [Glorot et al., 2010](http://proceedings.mlr.press/v9/glorot10a.html)
 |      ([pdf](http://jmlr.org/proceedings/papers/v9/glorot10a/glorot10a.pdf))
 |  
 |  Method resolution order:
 |      GlorotUniform
 |      VarianceScaling
 |      In

In [8]:
batch_size = 32

In [9]:
inputs = tf.placeholder(shape = [batch_size, X_train.shape[1]], dtype = fl32)
labels = tf.placeholder(shape = [batch_size, Y_train.shape[1]], dtype = fl32)

In [21]:
printoni(dirt(tf.initializers), 4)

constant         identity        random_normal        variance_scaling 
global_variables lecun_normal    random_uniform       zeros            
glorot_normal    lecun_uniform   tables_initializer                    
glorot_uniform   local_variables truncated_normal                      
he_normal        ones            uniform_unit_scaling                  
he_uniform       orthogonal      variables                             


In [39]:
with tf.variable_scope('l1', reuse = tf.AUTO_REUSE):
    w1 = tf.get_variable('w1', [inputs.shape[1].value, 110], initializer = he_uniform())
    b1 = tf.get_variable('b1', [110,], initializer = tf.initializers.zeros)
    layer1 = dropout( relu(add(matmul(inputs, w1), b1)), rate = 0.1 )

with tf.variable_scope('l2', reuse = tf.AUTO_REUSE):
    w2 = tf.get_variable('w2', [layer1.shape[1].value, 220], initializer = he_uniform())
    b2 = tf.get_variable('b2', [220,], initializer = tf.initializers.zeros)
    layer2 = dropout( relu(add(matmul(layer1, w2), b2)), rate = 0.1)

with tf.variable_scope('l3', reuse = tf.AUTO_REUSE):
    w3 = tf.get_variable('w3', [layer2.shape[1].value, 220], initializer = he_uniform())
    b3 = tf.get_variable('b3', [220,], initializer = tf.initializers.zeros)
    layer3 = dropout( relu(add(matmul(layer2, w3), b3)), rate = 0.1)

with tf.variable_scope('output_layer', reuse = tf.AUTO_REUSE):
    w4 = tf.get_variable('w4', [layer2.shape[1].value, 80], initializer = glorot_uniform())
    b4 = tf.get_variable('b3', [80,], initializer = tf.initializers.zeros)
    layer4 = add(matmul(layer3, w4), b4)

In [41]:
with tf.variable_scope('loss', reuse = tf.AUTO_REUSE):
    loss = tf.reduce_mean(tf.squared_difference(layer4, labels))
    loss_minimize = tf.train.AdamOptimizer().minimize(loss)

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for epoch in range(20):
    for i in range(X_train.shape[0] // batch_size):
        sess.run([loss, loss_minimize], 
                 feed_dict = {inputs: X_train[i * batch_size:(i + 1)*batch_size, :],
                              labels: Y_train[i * batch_size:(i + 1)*batch_size, :]})

    test_loss = []   
    for i in range(X_val.shape[0] // batch_size):
        new_loss,_ = sess.run([loss, loss_minimize], 
                 feed_dict = {inputs: X_val[i * batch_size:(i + 1)*batch_size, :],
                              labels: Y_val[i * batch_size:(i + 1)*batch_size, :]})
        test_loss.append(new_loss)

    import numpy as np
    mean_test_loss = np.array(test_loss).mean()
    print('Epoch: ' + str(epoch) + '  -  val_loss: ' + str(mean_test_loss))

print(sess.graph)
sess.close()

Epoch: 0  -  val_loss: 0.016354427
Epoch: 1  -  val_loss: 0.014080174
Epoch: 2  -  val_loss: 0.01342251
Epoch: 3  -  val_loss: 0.013251232
Epoch: 4  -  val_loss: 0.013097901
Epoch: 5  -  val_loss: 0.0130063845
Epoch: 6  -  val_loss: 0.01294462
Epoch: 7  -  val_loss: 0.012827262


In [109]:
mean_test_loss

0.013327003

In [115]:
mean_test_loss

0.0135560045

In [124]:
mean_test_loss

0.012763917

In [33]:
mean_test_loss # 20 epochs

0.0126214735

<h2>Save Model</h2>

In [119]:
tf.io.write_graph(sess.graph, '/tmp/my-model', 'train.pbtxt')

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4205122772179942051, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 8769741372308327298
 physical_device_desc: "device: XLA_CPU device"]